In [28]:
import sys
import os

# Add the project root directory to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

In [29]:
%pwd

'c:\\Users\\sylla\\Desktop\\MLOPS'

In [30]:
os.chdir('../')
%pwd

'c:\\Users\\sylla\\Desktop'

In [31]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionconfig :
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    def __init__(self, root_dir: Path, source_URL: str, local_data_file: Path, unzip_dir: Path):
        self.root_dir = root_dir
        self.source_URL = source_URL
        self.local_data_file = local_data_file
        self.unzip_dir = unzip_dir

In [32]:
from src.Project1_Data_Science.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.Project1_Data_Science.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath: str = CONFIG_FILE_PATH,
                 params_filepath: str = PARAMS_FILE_PATH,
                 schema_filepath: str = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        print(f"Type of artifacts_root: {type(self.config.artifacts_root)}")
        print(f"Value of artifacts_root: {self.config.artifacts_root}")


        create_directories([self.config.artifacts_root])

    def get_ingestion_config(self) -> DataIngestionconfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        dataingestion_config = DataIngestionconfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return dataingestion_config


In [34]:
import requests
import urllib.request  as request
from src.Project1_Data_Science import logger
import zipfile


In [35]:
## Component data Ingestion

class DataIngestion:
    def __init__(self, config: DataIngestionconfig):
        self.config = config
    # downloading the zip file from the source URL
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                filename, header =request.urlretrieve(
                    self.config.source_URL, 
                    self.config.local_data_file)
                logger.info(f"{filename} downloaded: \n  {header}")
            except Exception as e:
                raise Exception(f"Error downloading {filename}: {e}")
        else :
            logger.info(f"File already exists: {self.config.local_data_file}")
    # extracting the zip file
    def extract_zip_file(self):
        """Extracts the zip file to the specified directory.
        If the directory does not exist, it will be created.
        function return none
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted files to {unzip_path}")     

In [36]:
try :
    config= ConfigurationManager()
    data_ingestion_config = config.get_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)   
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

Lecture de YAML depuis C:\Users\sylla\Desktop\MLOPS\Project1_with_deployment\Project1_Data_Science\config\config.yaml
Lecture de YAML depuis C:\Users\sylla\Desktop\MLOPS\Project1_with_deployment\Project1_Data_Science\params.yaml
Lecture de YAML depuis C:\Users\sylla\Desktop\MLOPS\Project1_with_deployment\Project1_Data_Science\schema.yaml
Type of artifacts_root: <class 'str'>
Value of artifacts_root: artifacts
2025-04-05 12:39:58,183 - Datasciencelogger - INFO - Directory already exists for artifacts
2025-04-05 12:39:58,186 - Datasciencelogger - INFO - Directory created for artifacts/data_ingestion


2025-04-05 12:39:59,485 - Datasciencelogger - INFO - artifacts/data_ingestion/data.zip downloaded: 
  Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 979B:15C75F:1857FEC:1BD1067:67F10595
Accept-Ranges: bytes
Date: Sat, 05 Apr 2025 10:39:59 GMT
Via: 1.1 varnish
X-Served-By: cache-par-lfpb1150038-PAR
X-Cache: HIT
X-Cache-Hits: 1
X-Timer: S1743849600.593243,VS0,VE1
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 69cc80eea8b827454fae308e1f5328bdc1596de1
Expires: Sat, 05 Apr 2025 10:44:59 GMT
Source-Age: 163


2025-04-05 12:39:59,485 - Datasciencelogger - 